# 🧪 ADK with A2A Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application that implements the Agent2Agent (A2A) protocol.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform a2a-sdk --upgrade

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys

# sys.path.append("../")

### Import libraries

In [ ]:
# ruff: noqa
import asyncio
import json
import os
import requests
import uuid

import vertexai
from a2a.types import (
    Message,
    MessageSendParams,
    Part,
    Role,
    SendStreamingMessageRequest,
    TextPart,
)
from IPython.display import Markdown, display
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService

from app.agent_engine_app import AgentEngineApp
from tests.helpers import (
    build_get_request,
    build_post_request,
    poll_task_completion,
)

### Initialize Vertex AI Client

In [ ]:
# Initialize the Vertex AI client
LOCATION = "us-central1"

client = vertexai.Client(
    location=LOCATION,
)

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [ ]:
# Set to None to auto-detect from ./deployment_metadata.json, or specify manually
# "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"
REASONING_ENGINE_ID = None

if REASONING_ENGINE_ID is None:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass

print(f"Using REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")
# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=REASONING_ENGINE_ID)

#### Fetch Agent Card

In [ ]:
remote_a2a_agent_card = await remote_agent_engine.handle_authenticated_agent_card()
print(f"Agent: {remote_a2a_agent_card.name}")
print(f"URL: {remote_a2a_agent_card.url}")
print(f"Skills: {[s.description for s in remote_a2a_agent_card.skills]}")
print(f"Examples: {[s.examples for s in remote_a2a_agent_card.skills][0]}")

#### Send Message

In [ ]:
# Send the message using A2A protocol
message_data = {
    "messageId": f"msg-{os.urandom(8).hex()}",
    "role": "user",
    "parts": [{"kind": "text", "text": "What is the weather in New York?"}],
}

response = await remote_agent_engine.on_message_send(**message_data)

# Extract task object from response
task_object = None
for chunk in response:
    if isinstance(chunk, tuple) and chunk and hasattr(chunk[0], "id"):
        task_object = chunk[0]
        break

# Get task_id
if task_object:
    task_id = task_object.id
    print(f"Task started: {task_id}")
else:
    print("Could not retrieve the task object from the response.")

#### Poll for response

In [ ]:
# Poll for task completion
max_attempts = 30
for attempt in range(max_attempts):
    result = await remote_agent_engine.on_get_task(id=task_id, historyLength=1)

    task_state = result.status.state if hasattr(result, "status") else None
    print(f"Attempt {attempt + 1}: {task_state}")

    if task_state == "completed":
        print("Task completed!")
        break
    elif task_state == "failed":
        print(f"Task failed: {result}")
        break

    await asyncio.sleep(1)

# Extract and display artifacts
if hasattr(result, "artifacts") and result.artifacts:
    for artifact in result.artifacts:
        if artifact.parts and hasattr(artifact.parts[0], "root"):
            text = getattr(artifact.parts[0].root, "text", None)
            if text:
                display(Markdown(f"**Answer**:\n {text}"))
            else:
                print("Could not extract text from artifact parts.")
        else:
            print("Could not extract text from artifact parts.")
else:
    print("No artifacts found in result")

#### Register Feedback

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

#### Set up local Agent Engine

In [ ]:
local_agent_engine = await AgentEngineApp.create(
    artifact_service_builder=lambda: InMemoryArtifactService(),
    session_service_builder=lambda: InMemorySessionService(),
)
local_agent_engine.set_up()

#### Verify Custom Method is Registered

In [ ]:
test = local_agent_engine.register_operations()
print(test)

#### Fetch Agent Card

In [ ]:
request = build_get_request(None)
response = await local_agent_engine.handle_authenticated_agent_card(
    request=request, context=None
)
print(response)

#### Send Message

In [ ]:
message_data = {
    "message": {
        "messageId": f"msg-{os.urandom(8).hex()}",
        "content": [{"text": "What is the weather in New York?"}],
        "role": "ROLE_USER",
    },
}
request = build_post_request(message_data)

response = await local_agent_engine.on_message_send(request=request, context=None)
print(response)

#### Poll for response

In [ ]:
task_id = response["task"]["id"]
print(f"The Task ID is: {task_id}")

# Poll for completion using helper
final_response = await poll_task_completion(local_agent_engine, task_id)

# Extract and display artifacts
for artifact in final_response["artifacts"]:
    if artifact["parts"] and "text" in artifact["parts"][0]:
        display(Markdown(f"**Answer**:\n {artifact['parts'][0]['text']}"))
    else:
        print("Could not extract text from artifact parts.")

#### Register Feedback

In [ ]:
local_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

## If you are using Cloud Run

### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [ ]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

Send a message using A2A protocol

In [ ]:
# Create A2A message request
message = Message(
    message_id=f"msg-user-{uuid.uuid4()}",
    role=Role.user,
    parts=[Part(root=TextPart(text="Hello! Weather in New York?"))],
)

request = SendStreamingMessageRequest(
    id=f"req-{uuid.uuid4()}",
    params=MessageSendParams(message=message),
)

# Set up headers with authentication
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

# Send the streaming request to the A2A endpoint
response = requests.post(
    f"{SERVICE_URL}/a2a/app/",
    headers=headers,
    json=request.model_dump(mode="json", exclude_none=True),
    stream=True,
    timeout=60,
)

print(f"Response status code: {response.status_code}")

# Parse streaming A2A responses
for line in response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

### Local Testing

> You can run the application locally via the `make local-backend` command.

Send a message to the local backend service using the A2A protocol and receive a streaming response.

In [ ]:
# Create A2A message request
message = Message(
    message_id=f"msg-user-{uuid.uuid4()}",
    role=Role.user,
    parts=[Part(root=TextPart(text="Hello! Weather in New York?"))],
)

request = SendStreamingMessageRequest(
    id=f"req-{uuid.uuid4()}",
    params=MessageSendParams(message=message),
)

# Set up headers
headers = {"Content-Type": "application/json"}

# Send the streaming request to the local A2A endpoint
response = requests.post(
    "http://127.0.0.1:8000/a2a/app/",
    headers=headers,
    json=request.model_dump(mode="json", exclude_none=True),
    stream=True,
    timeout=60,
)

print(f"Response status code: {response.status_code}")

# Parse streaming A2A responses
for line in response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")